# Wrangling  OpenStreetMap Data  with MongoDB
### Duc Vu

OpenStreetMap is an open project that lets eveyone use and create a free editable map of the world.

## 1. Chosen Map Area

In this project, I choose to analyze data from Boston, Massachusetts want to show you to fix one type of error, that is the address of the street. And not only that, I also will show you how to put the data that has been audited into MongoDB instance. We also use MongoDB's Agregation Framework to get overview and analysis of the data

In [2]:
from IPython.display import HTML
HTML('<iframe width="425" height="350" frameborder="0" scrolling="no" marginheight="0" marginwidth="0" \
src="http://www.openstreetmap.org/export/embed.html?bbox=-71.442,42.1858,-70.6984,42.4918&amp;layer=mapnik"></iframe><br/>')

The dataset is here https://s3.amazonaws.com/metro-extracts.mapzen.com/boston_massachusetts.osm.bz2

In [4]:
filename = 'boston_massachusetts.osm'

I used the Overpass API to download the OpenStreetMap XML for the corresponding bounding box:

## 2. Auditing the Data

 In this project, I will parse through the downloaded OSM XML file with ElementTree and find the number of each type of element since the XML file are too large to work with in memory.

In [5]:
import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
        tags = {}
        for event, elem in ET.iterparse(filename):
            if elem.tag in tags:
                tags[elem.tag] +=1
            else:
                tags[elem.tag]= 1
                
        return tags
#tags = count_tags('chicago_illinois.osm')
tags = count_tags(filename)
pprint.pprint(tags)

{'bounds': 1,
 'member': 9586,
 'nd': 2242045,
 'node': 1886391,
 'osm': 1,
 'relation': 1186,
 'tag': 846441,
 'way': 294246}


Before processing the data and add it into MongoDB, I should check the "k" value for each "<tag>" and see if they can be valid keys in MongoDB, as well as see if there are any other potential problems.

I have built 3 regular expressions to check for certain patterns in the tags to change the data model
and expand the "addr:street" type of keys to a dictionary like this:


Here are three regular expressions: lower, lower_colon, and problemchars.

- lower: matches strings containing lower case characters
- lower_colon: matches strings containing lower case characters and a single colon within the string
- problemchars: matches characters that cannot be used within keys in MongoDB
 
 
- example: {"address": {"street": "Some value"}}

So, we have to see if we have such tags, and if we have any tags with problematic characters.
Please complete the function 'key_type'.

In [6]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":

        if lower.search(element.attrib['k']):
            keys["lower"] += 1
        elif lower_colon.search(element.attrib['k']):
            keys["lower_colon"] += 1
        elif problemchars.search(element.attrib['k']):
            keys["problemchars"] +=1
        else:
            keys["other"] +=1
        

    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


keys = process_map(filename)
pprint.pprint(keys)

{'lower': 749756, 'lower_colon': 56532, 'other': 40149, 'problemchars': 4}


Now I will redefine process_map to build a set of unique userid's found within the XML. I will then output the length of this set, representing the number of unique users making edits in the chosen map area.

In [7]:
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        #print element.attrib
        
        try:
            users.add(element.attrib['uid'])
        except KeyError:
            continue
        '''
        if "uid" in element.attrib:
            users.add(element.attrib['uid'])
        '''
    return users

users = process_map(filename)
#pprint.pprint(users)
print len(users)

1016


## 3. Problems Encountered in the Map

The majority of this project will be devoted to auditing and cleaning street names in the OSM XML file by changing the variable 'mapping' to reflect the changes needed to fix the unexpected or abbreviated street types to the appropriate ones in the expected list. I will find these abbreviations and replace them with their full text form. 

In [8]:
from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

In [9]:
def audit_street_type(street_types, street_name, rex):
    #m = street_type_re.search(street_name)
    m = rex.search(street_name)
    #print m
    #print m.group()
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


In [10]:
def audit(osmfile,rex):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'],rex)

    return street_types


In [11]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [12]:
st_types = audit(filename, rex = street_type_re)
pprint.pprint(dict(st_types))

{'1100': set(['First Street, Suite 1100']),
 '1702': set(['Franklin Street, Suite 1702']),
 '303': set(['First Street, Suite 303']),
 '6': set(['South Station, near Track 6']),
 '846028': set(['PO Box 846028']),
 'Artery': set(['Southern Artery']),
 'Ave': set(['360 Huntington Ave',
             '738 Commonwealth Ave',
             'Blue Hill Ave',
             'Boston Ave',
             'College Ave',
             'Commonwealth Ave',
             'Concord Ave',
             'Francesca Ave',
             'Harrison Ave',
             'Highland Ave',
             'Huntington Ave',
             'Josephine Ave',
             'Lexington Ave',
             'Massachusetts Ave',
             'Morrison Ave',
             'Mystic Ave',
             'Sagamore Ave',
             'Somerville Ave',
             "St. Paul's Ave",
             'Washington Ave',
             'Western Ave',
             'Willow Ave']),
 'Ave.': set(['Brighton Ave.',
              'Huntington Ave.',
              'Massac

In [13]:
# UPDATE THIS VARIABLE
mapping = { "ave" : "Avenue",
            "Ave" : "Avenue",
            "Ave.": "Avenue",
            "Ct" : "Court",
            "HIghway": "Highway",
            "Hwy": "Highway",
            "LEVEL": "Level",
            "Pkwy": "Parkway",
            "Pl": "Place",
            "rd." : "Road",
            "Rd" : "Road",
            "Rd." : "Road",
            "Sq." : "Square", 
            "st": "Street",
            "St": "Street",
            "St.": "Street",
            "St,": "Street", 
            "ST": "Street",
            "Street." : "Street",               
            }

In [14]:
def update_name(name, mapping,rex):

    #m = street_type_re.search(name)
    m = rex.search(name)
    if m:
        street_type = m.group()
        new_street_type = mapping[street_type]
        name = re.sub(rex, new_street_type, name) # re.sub(old_pattern, new_pattern, file)
        #name = street_type_re.sub(new_street_type, name)
    return name

In [15]:
for st_type, ways in st_types.iteritems():
    if st_type in mapping:
        for name in ways:
            better_name = update_name(name, mapping, rex = street_type_re)
            print name, "=>", better_name

Walnut St, => Walnut Street
Maverick St. => Maverick Street
Pearl St. => Pearl Street
Banks St. => Banks Street
Tremont St. => Tremont Street
Centre St. => Centre Street
Marshall St. => Marshall Street
Prospect St. => Prospect Street
Main St. => Main Street
Albion St. => Albion Street
Saint Mary's St. => Saint Mary's Street
Boylston St. => Boylston Street
Stuart St. => Stuart Street
Elm St. => Elm Street
Brookline St. => Brookline Street
Oakland Rd => Oakland Road
Abby Rd => Abby Road
Bristol Rd => Bristol Road
Squanto Rd => Squanto Road
Goodnough Rd => Goodnough Road
Soldiers Field Rd => Soldiers Field Road
Aberdeen Rd => Aberdeen Road
Massachusetts ave => Massachusetts Avenue
Newton ST => Newton Street
Longfellow Pl => Longfellow Place
Hancock Street. => Hancock Street
Monsignor O'Brien Hwy => Monsignor O'Brien Highway
Corey rd. => Corey Road
Brighton Ave. => Brighton Avenue
Spaulding Ave. => Spaulding Avenue
Massachusetts Ave. => Massachusetts Avenue
Somerville Ave. => Somerville Av

Finally, I will traverse the cardinal_directions dictionary and apply the updates for both street type and cardinal direction

In [16]:
cardinal_dir_re = re.compile(r'^[NSEW]\b\.?', re.IGNORECASE)

In [17]:
dir_st_types = audit(filename, rex = cardinal_dir_re)
pprint.pprint(dict(dir_st_types))

{'E': set(['E 4th St', 'E Elm Avenue']), 'N': set(['N Beacon St'])}


In [18]:
cardinal_directions_mapping = \
    {
        "E" : "East",
        "N" : "North",
        "S" : "South",
        "W" : "West"
    }

In [19]:
for st_type, ways in dir_st_types.iteritems():
    if st_type in cardinal_directions_mapping:
        for name in ways:
            better_name = update_name(name, cardinal_directions_mapping, rex = cardinal_dir_re)
            print name, "=>", better_name

E Elm Avenue => East Elm Avenue
E 4th St => East 4th St
N Beacon St => North Beacon St


Then I will count the total number of nodes and ways that contain a tag child with k="addr:street"

In [203]:
osm_file = open(filename, "r")
address_count = 0

for event, elem in ET.iterparse(osm_file, events=("start",)):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"): 
            if is_street_name(tag):
                address_count += 1

address_count

2976

## 4. Preparing for MongoDB

Before importing the XML data into MongoDB, I have to transform the shape of data into json documents structured (a list of dictionaries) like this


{

"id": "2406124091",

"type: "node",

"visible":"true",

"created": {
          
          "version":"2",
          
          "changeset":"17206049",
          
          "timestamp":"2013-08-03T16:43:42Z",
          
          "user":"linuxUser16",
          
          "uid":"1219059"
        
        },

"pos": [41.9757030, -87.6921867],

"address": {
          
          "housenumber": "5157",
          
          "postcode": "60625",
          
          "street": "North Lincoln Ave"
        
        },

"amenity": "restaurant",

"cuisine": "mexican",

"name": "La Cabana De Don Luis",

"phone": "1 (773)-271-5176"

}

Here are the rules:

- process only 2 types of top level tags: "node" and "way"

- all attributes of "node" and "way" should be turned into regular key/value pairs, except:

    - attributes in the CREATED array should be added under a key "created"
    
    - attributes for latitude and longitude should be added to a "pos" array,
      for use in geospacial indexing. Make sure the values inside "pos" array are floats
      and not strings.
- if second level tag "k" value contains problematic characters, it should be ignored
- if second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
- if second level tag "k" value does not start with "addr:", but contains ":", you can process it
  same as any other tag.
- if there is a second ":" that separates the type/direction of a street,
  the tag should be ignored, for example:

<tag k="addr:housenumber" v="5158"/>
<tag k="addr:street" v="North Lincoln Avenue"/>
<tag k="addr:street:name" v="Lincoln"/>
<tag k="addr:street:prefix" v="North"/>
<tag k="addr:street:type" v="Avenue"/>
<tag k="amenity" v="pharmacy"/>

  should be turned into:

{...

"address": {    
    
    "housenumber": 5158,
    
    "street": "North Lincoln Avenue"

}

"amenity": "pharmacy",

...

}

- for "way" specifically:

  <nd ref="305896090"/>  
  <nd ref="1719825889"/>

should be turned into

"node_refs": ["305896090", "1719825889"]

In [21]:
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

def shape_element(element):
    node = {}
    # create an address dictionary
    address = {}
    
    if element.tag == "node" or element.tag == "way" :
        # YOUR CODE HERE
        node["type"] = element.tag

        #for key in element.attrib.keys()
        for key in element.attrib:
            #print key

            if key in CREATED:
                if "created" not in node:
                    node["created"] = {}
                node["created"][key] = element.attrib[key]

            elif key in ["lat","lon"]:
                if "pos" not in node:
                    node["pos"] = [None, None]
                if key == "lat":
                    node["pos"][0] = float(element.attrib[key])
                elif key == "lon":
                    node["pos"][1] = float(element.attrib[key])
            else:
                node[key] = element.attrib[key]

            for tag in element.iter("tag"):
                tag_key = tag.attrib["k"]   # key
                tag_value = tag.attrib["v"] # value
                if not problemchars.match(tag_key):
                    if tag_key.startswith("addr:"):# Single colon beginning with addr
                        if "address" not in node:
                            node["address"] = {}
                        sub_addr = tag_key[len("addr:"):]
                        if not lower_colon.match(sub_addr): # Tags with no colon
                            address[sub_addr] = tag_value 
                            node["address"] = address
                            #node["address"][sub_addr] = tag_value
                    elif lower_colon.match(tag_key): # Single colon not beginnning with "addr:"
                        node[tag_key] = tag_value
                    else:
                        node[tag_key] = tag_value # Tags with no colon, not beginnning with "addr:"

        for nd in element.iter("nd"):
            if "node_refs" not in node:
                node["node_refs"] = []
            node["node_refs"].append(nd.attrib["ref"])

        return node
    else:
        return None

In [22]:
import codecs
import json

def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

process_map(filename)

[{'created': {'changeset': '244358',
   'timestamp': '2007-03-24T19:38:02Z',
   'uid': '6817',
   'user': 'lurker',
   'version': '1'},
  'created_by': 'YahooApplet 1.0',
  'id': '26746680',
  'pos': [42.3089253, -71.1191797],
  'type': 'node'},
 {'created': {'changeset': '14335103',
   'timestamp': '2012-12-19T19:24:31Z',
   'uid': '326503',
   'user': 'wambag',
   'version': '2'},
  'id': '30730952',
  'pos': [42.3678097, -71.0218711],
  'type': 'node'},
 {'created': {'changeset': '14335103',
   'timestamp': '2012-12-19T19:24:31Z',
   'uid': '326503',
   'user': 'wambag',
   'version': '2'},
  'id': '30730953',
  'pos': [42.3677364, -71.0218568],
  'type': 'node'},
 {'created': {'changeset': '14335103',
   'timestamp': '2012-12-19T19:24:31Z',
   'uid': '326503',
   'user': 'wambag',
   'version': '2'},
  'id': '30730954',
  'pos': [42.3676084, -71.0218168],
  'type': 'node'},
 {'created': {'changeset': '14335103',
   'timestamp': '2012-12-19T19:24:32Z',
   'uid': '326503',
   'user':

## 5. Data Overview

Check the size of XML and JSON files

In [194]:
import os
print "The downloaded XML file is {} MB".format(os.path.getsize(filename)/1.0e6) # convert from bytes to megabytes

The downloaded XML file is 421.230253 MB


In [195]:
print "The json file is {} MB".format(os.path.getsize(filename + ".json")/1.0e6) # convert from bytes to megabytes

The json file is 484.237902 MB


#### Execute mongod to run MongoDB
Use the subprocess module to run shell commands.

In [196]:
import signal
import subprocess

# The os.setsid() is passed in the argument preexec_fn so
# it's run after the fork() and before  exec() to run the shell.
pro = subprocess.Popen("mongod", preexec_fn = os.setsid) 

http://sharats.me/the-ever-useful-and-neat-subprocess-module.html

#### Connect database with pymongo

In [26]:
from pymongo import MongoClient

db_name = "osm"

client = MongoClient('localhost:27017')
db = client[db_name]

When we have to import a large amounts of data, mongoimport is recommended.

First build a mongoimport command, then use subprocess.call to execute

In [27]:
# Build mongoimport command
collection = filename[:filename.find(".")]
print collection
working_directory = "/Users/ducvu/Documents/ud032-master/final_project/"

json_file = filename + ".json"
print json_file
mongoimport_cmd = "mongoimport --db " + db_name + \
                  " --collection " + collection + \
                  " --file " + working_directory + json_file
print mongoimport_cmd 

# Before importing, drop collection if it exists
if collection in db.collection_names():
    print "dropping collection"
    db[collection].drop()

# Execute the command
print "Executing: " + mongoimport_cmd
subprocess.call(mongoimport_cmd.split())

boston_massachusetts
boston_massachusetts.osm.json
mongoimport --db osm --collection boston_massachusetts --file /Users/ducvu/Documents/ud032-master/final_project/boston_massachusetts.osm.json
dropping collection
Executing: mongoimport --db osm --collection boston_massachusetts --file /Users/ducvu/Documents/ud032-master/final_project/boston_massachusetts.osm.json


0

### Get the collection from the database

In [84]:
boston_db = db[collection]

### Number of Documents

In [85]:
boston_db.find().count()

2180637

### Number of Unique Users

In [86]:
len(boston_db.distinct('created.user'))

1001

### Number of Nodes and Ways

In [190]:
node_way = boston_db.aggregate([
        {"$group" : {"_id" : "$type", "count" : {"$sum" : 1}}}])

pprint.pprint(list(node_way))

[{u'_id': u'multipolygon', u'count': 1},
 {u'_id': u'video', u'count': 1},
 {u'_id': u'chain_link', u'count': 1},
 {u'_id': u'way', u'count': 294189},
 {u'_id': u'Collaborative Program', u'count': 6},
 {u'_id': u'charter', u'count': 1},
 {u'_id': u'special', u'count': 1},
 {u'_id': u'Approved Special Education', u'count': 12},
 {u'_id': u'Academic', u'count': 34},
 {u'_id': u'Special-Law', u'count': 3},
 {u'_id': u'civil', u'count': 1},
 {u'_id': u'node', u'count': 1885954},
 {u'_id': u'Special', u'count': 49},
 {u'_id': u'Charter', u'count': 17},
 {u'_id': u'Public', u'count': 182},
 {u'_id': u'broad_leaved', u'count': 4},
 {u'_id': u'School', u'count': 77},
 {u'_id': u'private', u'count': 3},
 {u'_id': u'County', u'count': 3},
 {u'_id': u'Private', u'count': 87},
 {u'_id': u'State', u'count': 2},
 {u'_id': u'Special-Medical', u'count': 7},
 {u'_id': u'Special-Institutional', u'count': 2}]


### Number of Nodes

In [90]:
boston_db.find({"type":"node"}).count()

1885954

### Number of Ways

In [91]:
boston_db.find({"type":"way"}).count()

294189

### Top Contributing User

In [191]:
top_user = boston_db.aggregate([
    {"$match":{"type":"node"}},
    {"$group":{"_id":"$created.user","count":{"$sum":1}}},
    {"$sort":{"count":-1}},
    {"$limit":1}
])

#print(list(top_user))
pprint.pprint(list(top_user))

[{u'_id': u'crschmidt', u'count': 1229402}]


### Number of users having only 1 post

In [168]:
type_buildings = boston_db.aggregate([
    {"$group":{"_id":"$created.user","count":{"$sum":1}}},
    {"$group":{"_id":{"postcount":"$count"},"num_users":{"$sum":1}}},
    {"$project":{"_id":0,"postcount":"$_id.postcount","num_users":1}},
    {"$sort":{"postcount":1}},
    {"$limit":1}
])

pprint.pprint(list(type_buildings))

[{u'num_users': 244, u'postcount': 1}]


### Number of Documents Containing a Street Address

In [197]:
boston_db.find({"address.street" : {"$exists" : 1}}).count()

3026

### Zip codes

In [179]:
zipcodes = boston_db.aggregate([
        {"$match" : {"address.postcode" : {"$exists" : 1}}}, \
        {"$group" : {"_id" : "$address.postcode", "count" : {"$sum" : 1}}}, \
        {"$sort" : {"count" : -1}}])

#for document in zipcodes:
#    print(document)
    
pprint.pprint(list(zipcodes))

[{u'_id': u'02135', u'count': 250},
 {u'_id': u'02139', u'count': 230},
 {u'_id': u'02144', u'count': 92},
 {u'_id': u'02215', u'count': 61},
 {u'_id': u'02114', u'count': 59},
 {u'_id': u'02143', u'count': 52},
 {u'_id': u'02169', u'count': 52},
 {u'_id': u'02134', u'count': 49},
 {u'_id': u'02138', u'count': 49},
 {u'_id': u'02130', u'count': 44},
 {u'_id': u'02116', u'count': 41},
 {u'_id': u'02446', u'count': 38},
 {u'_id': u'02142', u'count': 31},
 {u'_id': u'02472', u'count': 30},
 {u'_id': u'02134-1307', u'count': 29},
 {u'_id': u'02210', u'count': 28},
 {u'_id': u'02467', u'count': 26},
 {u'_id': u'02155', u'count': 23},
 {u'_id': u'02474', u'count': 22},
 {u'_id': u'02128', u'count': 22},
 {u'_id': u'02145', u'count': 20},
 {u'_id': u'02132', u'count': 17},
 {u'_id': u'02141', u'count': 16},
 {u'_id': u'02111', u'count': 16},
 {u'_id': u'02149', u'count': 15},
 {u'_id': u'02140', u'count': 14},
 {u'_id': u'02108', u'count': 11},
 {u'_id': u'02134-1433', u'count': 11},
 {u'_id'

### Top 5 Most Common Cities

In [201]:
cities = boston_db.aggregate([{"$match" : {"address.city" : {"$exists" : 1}}}, \
                           {"$group" : {"_id" : "$address.city", "count" : {"$sum" : 1}}}, \
                           {"$sort" : {"count" : -1}}, \
                           {"$limit" : 5}])

#for city in cities :
#    print city
    
pprint.pprint(list(cities))

[{u'_id': u'Boston', u'count': 594},
 {u'_id': u'Malden', u'count': 413},
 {u'_id': u'Cambridge', u'count': 323},
 {u'_id': u'Somerville', u'count': 153},
 {u'_id': u'Quincy', u'count': 51}]


### Top 10 Amenities

In [192]:
amenities = boston_db.aggregate([
        {"$match" : {"amenity" : {"$exists" : 1}}}, \
        {"$group" : {"_id" : "$amenity", "count" : {"$sum" : 1}}}, \
        {"$sort" : {"count" : -1}}, \
        {"$limit" : 10}])

#for document in amenities:
#    print document

pprint.pprint(list(amenities))

[{u'_id': u'parking', u'count': 1192},
 {u'_id': u'bench', u'count': 946},
 {u'_id': u'school', u'count': 774},
 {u'_id': u'restaurant', u'count': 532},
 {u'_id': u'parking_space', u'count': 444},
 {u'_id': u'place_of_worship', u'count': 407},
 {u'_id': u'library', u'count': 342},
 {u'_id': u'bicycle_parking', u'count': 238},
 {u'_id': u'cafe', u'count': 199},
 {u'_id': u'fast_food', u'count': 169}]


In [198]:
amenities = boston_db.aggregate([
    {"$match":{"amenity":{"$exists":1},"type":"node"}},
    {"$group":{"_id":"$amenity","count":{"$sum":1}}},
    {"$sort":{"count":-1}},
    {"$limit":10}
])

pprint.pprint(list(amenities))

[{u'_id': u'bench', u'count': 938},
 {u'_id': u'restaurant', u'count': 493},
 {u'_id': u'school', u'count': 370},
 {u'_id': u'place_of_worship', u'count': 317},
 {u'_id': u'bicycle_parking', u'count': 238},
 {u'_id': u'cafe', u'count': 182},
 {u'_id': u'fast_food', u'count': 154},
 {u'_id': u'library', u'count': 141},
 {u'_id': u'bicycle_rental', u'count': 141},
 {u'_id': u'fire_station', u'count': 96}]


### Most common building types

In [173]:
type_buildings = boston_db.aggregate([
    {'$match': {'building': {'$exists': 1}}}, 
    {'$group': { '_id': '$building','count': {'$sum': 1}}},
    {'$sort': {'count': -1}}, {'$limit': 20}
])

pprint.pprint(list(type_buildings))

[{u'_id': u'yes', u'count': 248378},
 {u'_id': u'garage', u'count': 673},
 {u'_id': u'house', u'count': 629},
 {u'_id': u'apartments', u'count': 422},
 {u'_id': u'university', u'count': 295},
 {u'_id': u'shed', u'count': 131},
 {u'_id': u'roof', u'count': 109},
 {u'_id': u'dormitory', u'count': 98},
 {u'_id': u'school', u'count': 88},
 {u'_id': u'residential', u'count': 76},
 {u'_id': u'commercial', u'count': 69},
 {u'_id': u'retail', u'count': 53},
 {u'_id': u'entrance', u'count': 37},
 {u'_id': u'storage_tank', u'count': 30},
 {u'_id': u'church', u'count': 27},
 {u'_id': u'home', u'count': 26},
 {u'_id': u'office', u'count': 20},
 {u'_id': u'industrial', u'count': 16},
 {u'_id': u'hospital', u'count': 7},
 {u'_id': u'hotel', u'count': 5}]


### Top Religions with Denominations

In [139]:
religions = boston_db.aggregate([
        {"$match" : {"amenity" : "place_of_worship"}}, \
        {"$group" : {"_id" : {"religion" : "$religion", "denomination" : "$denomination"}, "count" : {"$sum" : 1}}}, \
        {"$sort" : {"count" : -1}}])

#for document in religions:
#    print document

pprint.pprint(list(religions))

[{u'_id': {u'religion': u'christian'}, u'count': 189},
 {u'_id': {u'denomination': u'baptist', u'religion': u'christian'},
  u'count': 53},
 {u'_id': {u'denomination': u'methodist', u'religion': u'christian'},
  u'count': 22},
 {u'_id': {u'denomination': u'catholic', u'religion': u'christian'},
  u'count': 22},
 {u'_id': {}, u'count': 16},
 {u'_id': {u'denomination': u'roman_catholic', u'religion': u'christian'},
  u'count': 10},
 {u'_id': {u'denomination': u'presbyterian', u'religion': u'christian'},
  u'count': 10},
 {u'_id': {u'denomination': u'lutheran', u'religion': u'christian'},
  u'count': 9},
 {u'_id': {u'religion': u'jewish'}, u'count': 9},
 {u'_id': {u'denomination': u'pentecostal', u'religion': u'christian'},
  u'count': 8},
 {u'_id': {u'denomination': u'episcopal', u'religion': u'christian'},
  u'count': 8},
 {u'_id': {u'religion': u'unitarian'}, u'count': 6},
 {u'_id': {u'denomination': u'orthodox', u'religion': u'christian'},
  u'count': 4},
 {u'_id': {u'religion': u'uni

### Top 10 Leisures

In [154]:
leisures = boston_db.aggregate([{"$match" : {"leisure" : {"$exists" : 1}}}, \
                           {"$group" : {"_id" : "$leisure", "count" : {"$sum" : 1}}}, \
                           {"$sort" : {"count" : -1}}, \
                           {"$limit" : 10}])

#for document in leisures:
#    print document

pprint.pprint(list(leisures))

[{u'_id': u'park', u'count': 750},
 {u'_id': u'pitch', u'count': 545},
 {u'_id': u'recreation_ground', u'count': 350},
 {u'_id': u'playground', u'count': 334},
 {u'_id': u'nature_reserve', u'count': 105},
 {u'_id': u'garden', u'count': 35},
 {u'_id': u'sports_centre', u'count': 33},
 {u'_id': u'picnic_table', u'count': 33},
 {u'_id': u'swimming_pool', u'count': 28},
 {u'_id': u'common', u'count': 19}]


### Top 15 Universities

In [202]:
universities = boston_db.aggregate([
        {"$match" : {"amenity" : "university"}}, \
        {"$group" : {"_id" : {"name" : "$name"}, "count" : {"$sum" : 1}}}, \
        {"$sort" : {"count" : -1}},
        {"$limit":15}
    ])

pprint.pprint(list(universities))

[{u'_id': {u'name': u'Boston University'}, u'count': 41},
 {u'_id': {u'name': u'Massachusetts Institute of Technology'}, u'count': 10},
 {u'_id': {u'name': u'Suffolk University'}, u'count': 8},
 {u'_id': {u'name': u'Harvard University'}, u'count': 6},
 {u'_id': {u'name': None}, u'count': 4},
 {u'_id': {u'name': u'University of Massachusetts Boston'}, u'count': 3},
 {u'_id': {u'name': u'Boston University Medical Campus'}, u'count': 3},
 {u'_id': {u'name': u'University Hall'}, u'count': 2},
 {u'_id': {u'name': u'Harvard Medical School'}, u'count': 2},
 {u'_id': {u'name': u'Benjamin Franklin Institute of Technology'},
  u'count': 2},
 {u'_id': {u'name': u'Littauer Center'}, u'count': 2},
 {u'_id': {u'name': u'Northeastern University'}, u'count': 2},
 {u'_id': {u'name': u'Boston College'}, u'count': 2},
 {u'_id': {u'name': u'Radcliffe Gym'}, u'count': 1},
 {u'_id': {u'name': u'Agassiz House'}, u'count': 1}]


### Top 10 Schools

In [147]:
schools = boston_db.aggregate([
        {"$match" : {"amenity" : "school"}}, \
        {"$group" : {"_id" : {"name" : "$name"}, "count" : {"$sum" : 1}}}, \
        {"$sort" : {"count" : -1}},
        {"$limit":10}
    ])

pprint.pprint(list(schools))

[{u'_id': {u'name': None}, u'count': 13},
 {u'_id': {u'name': u'Milton Academy'}, u'count': 4},
 {u'_id': {u'name': u'Lincoln School'}, u'count': 3},
 {u'_id': {u'name': u'Boston Community Leadership Academy'}, u'count': 3},
 {u'_id': {u'name': u'New Mission High School'}, u'count': 3},
 {u'_id': {u'name': u'Dexter School'}, u'count': 3},
 {u'_id': {u'name': u'Boston Middle School Academy'}, u'count': 2},
 {u'_id': {u'name': u'Clark Avenue School'}, u'count': 2},
 {u'_id': {u'name': u'Kennedy Day School'}, u'count': 2},
 {u'_id': {u'name': u'Phillips School'}, u'count': 2}]


### Top Prisons 

In [153]:
prisons = boston_db.aggregate([
        {"$match" : {"amenity" : "prison"}}, \
        {"$group" : {"_id" : {"name" : "$name"}, "count" : {"$sum" : 1}}}, \
        {"$sort" : {"count" : -1}}])

pprint.pprint(list(prisons))

[{u'_id': {u'name': u'Norfolk County Jail'}, u'count': 1},
 {u'_id': {u'name': u'Middlesex County Jail (Cambridge)'}, u'count': 1},
 {u'_id': {u'name': u'Suffolk County House of Correction'}, u'count': 1},
 {u'_id': {u'name': u'Suffolk County Jail'}, u'count': 1},
 {u'_id': {u'name': u'Boston Pre-Release Center'}, u'count': 1},
 {u'_id': {u'name': u'Lemuel Shattuck Hospital Correctional Unit'},
  u'count': 1}]


### Top 10 Hospitals

In [189]:
hospitals = boston_db.aggregate([
        {"$match" : {"amenity" : "hospital"}}, \
        {"$group" : {"_id" : {"name" : "$name"}, "count" : {"$sum" : 1}}}, \
        {"$sort" : {"count" : -1}},
        {"$limit":10}
    ])


pprint.pprint(list(hospitals))

[{u'_id': {u'name': u'Carney Hospital'}, u'count': 3},
 {u'_id': {u'name': u'Steward Satellite Emergency Facility - Quincy'},
  u'count': 2},
 {u'_id': {u'name': u"St. Elizabeth's Medical Center"}, u'count': 2},
 {u'_id': {u'name': u'Arbour Hospital'}, u'count': 2},
 {u'_id': {u'name': u'Bournewood Hospital'}, u'count': 2},
 {u'_id': {u'name': u'Arbour-Hri Hospital'}, u'count': 2},
 {u'_id': {u'name': u'Cambridge Health Alliance-Whidden Memorial Hospital'},
  u'count': 2},
 {u'_id': {u'name': u'Faulkner Hospital'}, u'count': 2},
 {u'_id': {u'name': u'Central Street Health Center'}, u'count': 2},
 {u'_id': {u'name': None}, u'count': 2}]


### Most popular cuisines in fast foods

In [169]:
fast_food = boston_db.aggregate([
    {"$match":{"cuisine":{"$exists":1},"amenity":"fast_food"}},
    {"$group":{"_id":"$cuisine","count":{"$sum":1}}},
    {"$sort":{"count":-1}},
    {"$limit":10}
])

pprint.pprint(list(fast_food))

[{u'_id': u'pizza', u'count': 27},
 {u'_id': u'sandwich', u'count': 24},
 {u'_id': u'burger', u'count': 22},
 {u'_id': u'mexican', u'count': 13},
 {u'_id': u'chinese', u'count': 5},
 {u'_id': u'chicken', u'count': 4},
 {u'_id': u'italian', u'count': 4},
 {u'_id': u'ice_cream', u'count': 4},
 {u'_id': u'coffee_shop', u'count': 3},
 {u'_id': u'kebab', u'count': 2}]


### Most popular gas station brands

In [170]:
gas_station_brands = boston_db.aggregate([
    {"$match":{"brand":{"$exists":1},"amenity":"fuel"}},
    {"$group":{"_id":"$brand","count":{"$sum":1}}},
    {"$sort":{"count":-1}},
    {"$limit":10}
])

pprint.pprint(list(gas_station_brands))

[{u'_id': u'Gulf', u'count': 4},
 {u'_id': u'Shell', u'count': 3},
 {u'_id': u'Hess', u'count': 3},
 {u'_id': u'Super Petroleum', u'count': 1},
 {u'_id': u"Eli's", u'count': 1},
 {u'_id': u'APrime Energy', u'count': 1},
 {u'_id': u'US Petroleum', u'count': 1},
 {u'_id': u'Cumberland Farm', u'count': 1},
 {u'_id': u'Valvoline Oil Change', u'count': 1},
 {u'_id': u'Sunoco', u'count': 1}]


### Most popular banks

In [171]:
banks = boston_db.aggregate([
    {"$match":{"name":{"$exists":1},"amenity":"bank"}},
    {"$group":{"_id":"$name","count":{"$sum":1}}},
    {"$sort":{"count":-1}},
    {"$limit":10}
])

pprint.pprint(list(banks))

[{u'_id': u'Bank of America', u'count': 11},
 {u'_id': u'Citizens Bank', u'count': 7},
 {u'_id': u'TD Bank', u'count': 6},
 {u'_id': u'Eastern Bank', u'count': 4},
 {u'_id': u'Cambridge Savings Bank', u'count': 4},
 {u'_id': u'Santander', u'count': 4},
 {u'_id': u'Sovereign Bank', u'count': 4},
 {u'_id': u'Brookline Bank', u'count': 3},
 {u'_id': u'East Cambridge Savings Bank', u'count': 2},
 {u'_id': u'Cambridge Trust Company', u'count': 2}]


### Most popular restaurants

In [172]:
restaurants = boston_db.aggregate([
    {"$match":{"name":{"$exists":1},"amenity":"restaurant"}},
    {"$group":{"_id":"$name","count":{"$sum":1}}},
    {"$sort":{"count":-1}},
    {"$limit":10}
])

pprint.pprint(list(restaurants))

[{u'_id': u'Panera Bread', u'count': 6},
 {u'_id': u"Bertucci's", u'count': 4},
 {u'_id': u"Crazy Dough's", u'count': 2},
 {u'_id': u'Olecito', u'count': 2},
 {u'_id': u'The Elephant Walk', u'count': 2},
 {u'_id': u'Chipotle', u'count': 2},
 {u'_id': u'Dunkin Donuts', u'count': 2},
 {u'_id': u'Boloco', u'count': 2},
 {u'_id': u'Ninety Nine', u'count': 2},
 {u'_id': u'Boca Grande', u'count': 2}]


## 6. Additional Ideas

Analyzing the data of Boston I found out that not all nodes or ways include this information since its geographical position is represented within regions of a city. What could be done in this case, is check if each node or way belongs to a city based on the latitude and longitude and ensure that the property "address.city" is properly informed. By doing so, we could get statistics related to cities in a much more reliable way. In fact, I think this is the biggest benefit to anticipate problems and implement improvements to the data you want to analyze. Real world data are very susceptible to being incomplete, noisy and inconsistent which means that if you have low-quality of data the results of their analysis will also be of poor quality.

  I think that extending this open source project to include data such as user reviews of establishments, subjective areas of what bound a good and bad neighborhood, housing price data, school reviews, walkability/bikeability, quality of mass transit, and on would form a solid foundation of robust recommender systems. These recommender systems could aid users in anything from finding a new home or apartment to helping a user decide where to spend a weekend afternoon.

Another alternative to help in the absence of information in the region would be the use of gamification to make more people help in the map contribution. Something like Waze app already does today.

### Additional data exploration using MongoDB queries

### Conclusion         

http://www.choskim.me/how-to-install-mongodb-on-apples-mac-os-x/

https://books.google.com/books?id=_VkrAQAAQBAJ&pg=PA241&lpg=PA241&dq=execute+mongodb+command+in+ipython&source=bl&ots=JqnwlwRvkN&sig=h-TrwspKAmHt1g1ELItnWkDmRHs&hl=en&sa=X&ved=0ahUKEwiJnaiikIrLAhUM8CYKHZ8mBrcQ6AEILzAD#v=onepage&q=execute%20mongodb%20command%20in%20ipython&f=false